In [1]:
import sys
import numpy as np
import pandas as pd

## Load DMS and store object

In [13]:
#sys.path.append('../')

from data_prep import dm_saver, data_manager

dms = dm_saver.DM_Saver.load('../dms')

dml = data_manager.Data_Manager()

dml.dms2dm(dms)

## Normalizing the train data and saving the amplitude

In [18]:
pv_a = dml.train_a['pv_measurement']
pv_b = dml.train_b['pv_measurement']
pv_c = dml.train_c['pv_measurement']

print('A:\n',pv_a.describe())
print('B:\n',pv_b.describe())
print('C:\n',pv_c.describe())

A:
 count    136517.000000
mean          1.000000
std           1.827595
min           0.000000
25%           0.000000
50%           0.020610
75%           1.103184
max           9.103887
Name: pv_measurement, dtype: float64
B:
 count    132097.000000
mean          1.000000
std           1.990548
min           0.000000
25%           0.000000
50%           0.000000
75%           1.002929
max          11.963511
Name: pv_measurement, dtype: float64
C:
 count    105661.000000
mean          1.000000
std           2.122202
min           0.000000
25%           0.000000
50%           0.000000
75%           0.795349
max          12.980092
Name: pv_measurement, dtype: float64


### Mean value

In [15]:
dml.amplitude = np.array([pv_a.mean(), pv_b.mean(),pv_c.mean()])

### Downscaling

In [16]:
dml.train_a['pv_measurement'] = pv_a / dml.amplitude[0]
dml.train_b['pv_measurement'] = pv_b / dml.amplitude[1]
dml.train_c['pv_measurement'] = pv_c / dml.amplitude[2]

In [17]:
%store dml

Stored 'dml' (Data_Manager)
